In [1]:
# !nvcc --version
# !nvidia-smi

import os, shutil
import numpy as np
import matplotlib.pyplot as plt
from cellpose import core, utils, io, models, metrics
from data_loader import CellDataset, split_dataset

use_GPU = core.use_gpu()
yn = ['NO', 'YES']
print(f'>>> GPU activated? {yn[use_GPU]}')

>>> GPU activated? YES


In [2]:
#@markdown ###Path to images and masks:

train_dir = "/media/balint/st1/data/cell-counting/annotated/" #@param {type:"string"}
test_dir = "" #@param {type:"string"}
model_dir = "./"
#Define where the patch file will be saved
base = "/content"

# model name and path
#@markdown ###Name of the pretrained model to start from and new model name:
from cellpose import models
initial_model = "cyto" #@param ['cyto','nuclei','tissuenet','livecell','cyto2','CP','CPx','TN1','TN2','TN3','LC1','LC2','LC3','LC4','scratch']
model_name = "single_cell_params_test.pt" #@param {type:"string"}

# other parameters for training.
#@markdown ###Training Parameters:
#@markdown Number of epochs:
n_epochs =  100#@param {type:"number"}

Channel_to_use_for_training = "Grayscale" #@param ["Grayscale", "Blue", "Green", "Red"]

# @markdown ###If you have a secondary channel that can be used for training, for instance nuclei, choose it here:

Second_training_channel= "None" #@param ["None", "Blue", "Green", "Red"]


#@markdown ###Advanced Parameters

Use_Default_Advanced_Parameters = True #@param {type:"boolean"}
#@markdown ###If not, please input:
learning_rate = 0.1 #@param {type:"number"}
weight_decay = 0.0001 #@param {type:"number"}

if (Use_Default_Advanced_Parameters): 
  print("Default advanced parameters enabled")
  learning_rate = 0.01 
  weight_decay = 0.0001
  
#here we check that no model with the same name already exist, if so delete
model_path = train_dir + 'models/'
if os.path.exists(model_path+'/'+model_name):
  print("!! WARNING: "+model_name+" already exists and will be deleted in the following cell !!")
  
if len(test_dir) == 0:
  test_dir = None

# Here we match the channel to number
if Channel_to_use_for_training == "Grayscale":
  chan = 0
elif Channel_to_use_for_training == "Blue":
  chan = 3
elif Channel_to_use_for_training == "Green":
  chan = 2
elif Channel_to_use_for_training == "Red":
  chan = 1


if Second_training_channel == "Blue":
  chan2 = 3
elif Second_training_channel == "Green":
  chan2 = 2
elif Second_training_channel == "Red":
  chan2 = 1
elif Second_training_channel == "None":
  chan2 = 0

if initial_model=='scratch':
  initial_model = 'None'

Default advanced parameters enabled


In [3]:
data_paths = []
with open("./file_paths_2.txt", "r") as fp:
    data_paths = fp.read().strip().split('\n')
data_paths = data_paths
data_paths

['/media/balint/st3/projects/data/classification_datasets/lognorm-images/fibronectin/20210526_MDAMB231_fn/A1',
 '/media/balint/st3/projects/data/classification_datasets/lognorm-images/fibronectin/20210526_MDAMB231_fn/A2',
 '/media/balint/st3/projects/data/classification_datasets/lognorm-images/fibronectin/20210526_MDAMB231_fn/A3',
 '/media/balint/st3/projects/data/classification_datasets/lognorm-images/fibronectin/20210526_MDAMB231_fn/A4',
 '/media/balint/st3/projects/data/classification_datasets/lognorm-images/fibronectin/20210526_MDAMB231_fn/B1',
 '/media/balint/st3/projects/data/classification_datasets/lognorm-images/fibronectin/20210526_MDAMB231_fn/B2',
 '/media/balint/st3/projects/data/classification_datasets/lognorm-images/fibronectin/20210526_MDAMB231_fn/B3',
 '/media/balint/st3/projects/data/classification_datasets/lognorm-images/fibronectin/20210526_MDAMB231_fn/B4',
 '/media/balint/st3/projects/data/classification_datasets/lognorm-images/fibronectin/20210526_MDAMB231_fn/C1',
 

In [4]:
train_paths, test_paths = split_dataset(data_paths, 0.25)
train_ds = CellDataset(paths=train_paths, generate_flows=True)
test_ds = CellDataset(paths=test_paths, generate_flows=True)

In [5]:
# start logger (to see training across epochs)
logger = io.logger_setup()

# DEFINE CELLPOSE MODEL (without size model)
model = models.CellposeModel(gpu=use_GPU, model_type=initial_model)

# set channels
channels = [chan, chan2]

new_model_path = model.train(train_ds, test_ds, save_path=model_dir, 
                              n_epochs=n_epochs,
                              learning_rate=learning_rate, 
                              weight_decay=weight_decay, 
                              nimg_per_epoch=5,
                              model_name=model_name,
                              batch_size=8
                              )

# diameter of labels in training images
diam_labels = model.diam_labels.copy()

2023-08-12 19:27:15,924 [INFO] WRITING LOG OUTPUT TO /home/balint/.cellpose/run.log
2023-08-12 19:27:15,925 [INFO] 
cellpose version: 	2.2 
platform:       	linux 
python version: 	3.8.15 
torch version:  	2.0.0+cu117
2023-08-12 19:27:15,928 [INFO] >> cyto << model set to be used
2023-08-12 19:27:15,932 [INFO] ** TORCH CUDA version installed and working. **
2023-08-12 19:27:15,933 [INFO] >>>> using GPU
2023-08-12 19:27:16,177 [INFO] >>>> model diam_mean =  30.000 (ROIs rescaled to this size during training)
2023-08-12 19:27:50,133 [INFO] >>>> median diameter set to = 30
2023-08-12 19:27:50,136 [INFO] >>>> mean of training label mask diameters (saved to model) 48.758
2023-08-12 19:27:50,142 [INFO] >>>> training network with 2 channel input <<<<
2023-08-12 19:27:50,143 [INFO] >>>> LR: 0.01000, batch_size: 8, weight_decay: 0.00010
2023-08-12 19:27:50,143 [INFO] >>>> ntrain = 36, ntest = 12
2023-08-12 19:27:50,144 [INFO] >>>> ntrain = 36
2023-08-12 19:27:50,145 [INFO] >>>> nimg_per_epoch =

## Evaluate on test data (optional)

If you have test data, check performance

In [ ]:
# get files (during training, test_data is transformed so we will load it again)
output = io.load_train_test_data(test_dir, mask_filter='_seg.npy')
test_data, test_labels = output[:2]

# run model on test images
masks = model.eval(test_data, 
                   channels=[chan, chan2],
                   diameter=diam_labels)[0]

# check performance using ground truth labels
ap = metrics.average_precision(test_labels, masks)[0]
print('')
print(f'>>> average precision at iou threshold 0.5 = {ap[:,0].mean():.3f}')


plot masks

In [ ]:

plt.figure(figsize=(12,8), dpi=150)
for k,im in enumerate(test_data):
    img = im.copy()
    plt.subplot(3,len(train_files), k+1)
    img = np.vstack((img, np.zeros_like(img)[:1]))
    img = img.transpose(1,2,0)
    plt.imshow(img)
    plt.axis('off')
    if k==0:
        plt.title('image')

    plt.subplot(3,len(train_files), len(train_files) + k+1)
    plt.imshow(masks[k])
    plt.axis('off')
    if k==0:
        plt.title('predicted labels')

    plt.subplot(3,len(train_files), 2*len(train_files) + k+1)
    plt.imshow(test_labels[k])
    plt.axis('off')
    if k==0:
        plt.title('true labels')
plt.tight_layout()